In [3]:
import osmnx as ox, networkx as nx, matplotlib.cm as cm, pandas as pd, numpy as np, geopandas as gpd
import matplotlib.pyplot as plt
from importlib import reload
import functools
import time
%matplotlib inline

pd.set_option('precision', 5)
pd.options.display.float_format = '{:20.2f}'.format
pd.set_option('display.float_format', lambda x: '%.3f' % x)
pd.options.mode.chained_assignment = None

In [4]:
import landmarks_functions as lf
import utilities as uf

In [90]:
reload(lf)

<module 'landmarks_functions' from 'C:\\Users\\g_filo01\\sciebo\\scripts\\Image of the City\\landmarks_functions.py'>

# Computing landmarkness on paths and nodes

In [5]:
#initialise path, names, etc.

city_name = 'London'
folder_ouptut ='Outputs/'+city_name+'/'
epsg = 27700
crs = {'init': 'epsg:27700', 'no_defs': True}

In [127]:
# try reading nodes, paths, landmarks and sight_lines
# At this point, every element has already been extracted

nodes = gpd.read_file(folder_ouptut+city_name+'_nodes.shp', driver='ESRI Shapefile')
paths = gpd.read_file(folder_ouptut+city_name+'_paths.shp', driver='ESRI Shapefile')
landmarks = gpd.read_file(folder_ouptut+city_name+'_landmarks.shp', driver='ESRI Shapefile')
sight_lines = gpd.read_file(folder_ouptut+city_name+'_sightlines.shp', driver='ESRI Shapefile')

## Computing Local salience landmarks and assigning scores to Nodes

In [91]:
%%time
nodes = lf.local_salience(nodes, landmarks)
nodes = lf.distant_landmarks(nodes, landmarks, sight_lines, smaller_area = False)

Wall time: 1min 53s


## Visibility Matrix

In [36]:
%%time
# nodes_2dvis_field = ic.advance_visibility_nodes(nodes, paths, landmarks, field_view = True)
nodes_2dvis = lf.advance_visibility_nodes(nodes, paths, landmarks, field_view = False)

progress: 99 %Wall time: 3h 21min 38s


In [37]:
visibility_matrix = lf.visibility_matrix(landmarks, nodes_2dvis, nodes)

In [38]:
visibility_matrix.to_csv(folder_ouptut+city_name+'_visibility_matrix.csv')

## Conversion from lists to string, to export the file

In [98]:
to_convert = ['loc_land', 'loc_scor', 'dist_land', 'dist_scor', 'anchors']
nodes_string = nodes.copy()
for column in to_convert: nodes_string[column] = nodes_string[column].astype(str)

In [99]:
nodes_string.to_file(folder_ouptut+city_name+'_nodes.shp', driver='ESRI Shapefile')

In [128]:
t = pd.read_csv(folder_ouptut+city_name+'_visibility_matrix.csv')

In [173]:
vm.to_csv(folder_ouptut+city_name+'_visibility_matrix.csv')

In [156]:
t.remove('buildingID')

In [159]:
results = list(map(int, t))

In [169]:
nodes.head()

,nodeID,x,y,height,weight,Bc,Rc400,Rc600,Bc400,Bc600,...,Rc400_sc,Rc600_sc,Bc400_sc,Bc600_sc,loc_land,loc_scor,dist_land,dist_scor,anchors,geometry
0,11,528303.896,182074.120,2,6.000,1137513.000,1980.000,4506.000,1423.000,407.000,...,0.100,0.134,0.034,0.024,"[25158, 27428, 27429, 25160]","[0.769, 0.477, 0.461, 0.434]",[27131],[0.612],None,POINT (528303.8957000002 182074.1204000004)
1,43,528704.146,182068.777,2,6.000,315137.000,2007.000,4080.000,3951.000,1315.000,...,0.102,0.121,0.095,0.079,"[25153, 27440]","[0.603, 0.542]",None,None,[25171],POINT (528704.1456000004 182068.7771000005)
2,44,528834.300,182006.134,2,35.000,1198585.000,2393.000,4766.000,4665.000,1410.000,...,0.121,0.142,0.112,0.085,"[27440, 25162, 25165]","[0.542, 0.401, 0.317]",None,None,"[25171, 27131]",POINT (528834.3003000002 182006.1341999993)
3,45,528610.774,181808.799,2,29.000,343667.000,2279.000,6341.000,3628.000,773.000,...,0.116,0.191,0.087,0.047,"[26889, 25142, 26871, 25141]","[0.641, 0.576, 0.502, 0.495]",[25171],[0.859],None,POINT (528610.7739000004 181808.7993000001)
4,46,528573.726,181929.185,2,51.000,1146568.000,2162.000,5616.000,5395.000,969.000,...,0.110,0.169,0.129,0.058,"[25142, 27435, 25141, 25168]","[0.576, 0.514, 0.495, 0.379]",None,None,None,POINT (528573.7255999995 181929.1849000007)


In [172]:
vm.head()

,buildingID,11,43,44,45,46,47,48,49,50,...,17343,17344,17345,17346,17347,17348,17349,17350,17352,17353
0,26977,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
1,26978,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
2,26979,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
3,26980,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
4,26981,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0


In [164]:
z = list(set(results) - set(s))

In [124]:
for i in range(0, len(s)):
    print(t[i], s[i])

10 10
11 11
43 43
44 44
45 45
46 46
47 47
48 48
49 49
50 50
51 51
52 52
53 53
54 54
55 55
56 56
57 57
58 58
59 59
60 60
61 61
62 62
63 63
64 64
65 65
66 66
69 69
70 70
71 71
72 72
73 73
74 74
75 75
81 81
82 82
83 83
84 84
85 85
86 86
87 87
88 88
89 89
90 90
91 91
92 92
94 94
95 95
96 96
101 101
104 104
119 119
120 120
132 132
173 173
174 174
175 175
176 176
177 177
178 178
179 179
180 180
181 181
182 182
183 183
184 184
185 185
186 186
187 187
188 188
189 189
190 190
200 200
201 201
202 202
203 203
204 204
206 206
207 207
208 208
209 209
210 210
211 211
212 212
213 213
216 216
217 217
218 218
219 219
220 220
221 221
222 222
223 223
224 224
225 225
226 226
227 227
228 228
229 229
230 230
231 231
232 232
233 233
234 234
235 235
236 236
237 237
238 238
239 239
240 240
241 241
242 242
243 243
244 244
245 245
246 246
247 247
248 248
249 249
250 250
251 251
252 252
253 253
254 254
255 255
256 256
257 257
258 258
259 259
260 260
261 261
262 262
263 263
264 264
265 265
266 266
267 267
268 268


2247 2247
2248 2248
2249 2249
2250 2250
2251 2251
2252 2252
2253 2253
2254 2254
2255 2255
2256 2256
2257 2257
2258 2258
2259 2259
2260 2260
2261 2261
2262 2262
2263 2263
2264 2264
2265 2265
2266 2266
2267 2267
2268 2268
2269 2269
2270 2270
2271 2271
2272 2272
2273 2273
2274 2274
2275 2275
2276 2276
2277 2277
2278 2278
2279 2279
2280 2280
2281 2281
2282 2282
2283 2283
2284 2284
2285 2285
2286 2286
2287 2287
2288 2288
2289 2289
2290 2290
2291 2291
2292 2292
2293 2293
2294 2294
2295 2295
2296 2296
2297 2297
2298 2298
2299 2299
2300 2300
2301 2301
2302 2302
2303 2303
2304 2304
2305 2305
2306 2306
2307 2307
2308 2308
2309 2309
2310 2310
2311 2311
2312 2312
2313 2313
2314 2314
2315 2315
2316 2316
2317 2317
2318 2318
2319 2319
2320 2320
2321 2321
2322 2322
2323 2323
2324 2324
2325 2325
2326 2326
2327 2327
2328 2328
2329 2329
2330 2330
2331 2331
2332 2332
2333 2333
2334 2334
2335 2335
2336 2336
2337 2337
2338 2338
2339 2339
2340 2340
2341 2341
2342 2342
2343 2343
2344 2344
2345 2345
2346 2346


4230 4230
4231 4231
4232 4232
4233 4233
4234 4234
4235 4235
4236 4236
4237 4237
4238 4238
4239 4239
4240 4240
4244 4244
4295 4295
4297 4297
4298 4298
4299 4299
4304 4304
4305 4305
4306 4306
4307 4307
4308 4308
4309 4309
4310 4310
4312 4312
4314 4314
4315 4315
4316 4316
4319 4319
4320 4320
4321 4321
4325 4325
4326 4326
4327 4327
4328 4328
4334 4334
4349 4349
4350 4350
4351 4351
4387 4387
4388 4388
4389 4389
4390 4390
4391 4391
4392 4392
4393 4393
4394 4394
4399 4399
4405 4405
4410 4410
4411 4411
4412 4412
4421 4421
4422 4422
4423 4423
4469 4469
4491 4491
4492 4492
4493 4493
4494 4494
4518 4518
4519 4519
4520 4520
4521 4521
4522 4522
4523 4523
4524 4524
4578 4578
4581 4581
4589 4589
4590 4590
4591 4591
4592 4592
4599 4599
4601 4601
4636 4636
4690 4690
4697 4697
4698 4698
4715 4715
4791 4791
4798 4798
4803 4803
4820 4820
4880 4880
4888 4888
4893 4893
4898 4898
4899 4899
4901 4901
4902 4902
4903 4903
4905 4905
4906 4906
4907 4907
4919 4919
4920 4920
4921 4921
4922 4922
4923 4923
4924 4924


10380 10380
10386 10386
10387 10387
10390 10390
10392 10392
10394 10394
10396 10396
10406 10406
10407 10407
10408 10408
10410 10410
10412 10412
10413 10413
10415 10415
10416 10416
10417 10417
10418 10418
10419 10419
10420 10420
10422 10422
10425 10425
10430 10430
10431 10431
10432 10432
10433 10433
10434 10434
10436 10436
10437 10437
10439 10439
10440 10440
10441 10441
10442 10442
10443 10443
10444 10444
10445 10445
10448 10448
10449 10449
10451 10451
10452 10452
10453 10453
10454 10454
10455 10455
10457 10457
10458 10458
10459 10459
10460 10460
10461 10461
10462 10462
10478 10478
10501 10501
10503 10503
10504 10504
10505 10505
10508 10508
10509 10509
10510 10510
10511 10511
10512 10512
10513 10513
10514 10514
10515 10515
10516 10516
10517 10517
10518 10518
10519 10519
10520 10520
10521 10521
10522 10522
10523 10523
10524 10524
10525 10525
10526 10526
10527 10527
10528 10528
10529 10529
10530 10530
10531 10531
10532 10532
10533 10533
10534 10534
10536 10536
10537 10537
10538 10538
1053

13854 13854
13855 13855
13856 13856
13857 13857
13858 13858
13859 13859
13860 13860
13861 13861
13862 13862
13863 13863
13864 13864
13865 13865
13866 13866
13867 13867
13868 13868
13869 13869
13870 13870
13872 13872
13878 13878
13880 13880
13881 13881
13882 13882
13883 13883
13884 13884
13885 13885
13886 13886
13887 13887
13888 13888
13889 13889
13890 13890
13891 13891
13892 13892
13893 13893
13894 13894
13897 13897
13898 13898
13899 13899
13900 13900
13901 13901
13902 13902
13903 13903
13904 13904
13905 13905
13906 13906
13907 13907
13908 13908
13909 13909
13914 13914
13915 13915
13916 13916
13917 13917
13918 13918
13919 13919
13921 13921
13935 13935
13937 13937
13938 13938
13942 13942
13943 13943
13944 13944
13945 13945
13947 13947
13948 13948
13949 13949
13950 13950
13951 13951
13952 13952
13953 13953
13954 13954
13955 13955
13956 13956
13957 13957
13958 13958
13959 13959
13960 13960
13961 13961
13962 13962
13963 13963
13964 13964
13965 13965
13966 13966
13967 13967
13968 13968
1396

# Computing edge betweenness 
## Primal graph

In [ ]:
nodes_graph = nodes[['x', 'y', 'nodeID', 'DD_sc', 'geometry']]
nodes_graph.gdf_name = 'Nodes_list' #for OSMNx

attributes_nodes = ['DD_sc', 'geometry']
edges_costs = ['length', 'length_sc']

G = ic.graph_fromGDF(nodes_graph, paths, attributes_nodes, edges_costs)

In [ ]:
from heapq import heappush, heappop
from itertools import count
from networkx import NetworkXError
import networkx as nx

def astar_path(G, source, target, nodes_gdf, landmarks_gdf, weight=None):
 
    if G.is_multigraph():
        raise NetworkXError("astar_path() not implemented for Multi(Di)Graphs")

    push = heappush
    pop = heappop
    sindex = landmarks_gdf.sindex
    index_lScore = landmarks_gdf.columns.get_loc("lScore")+1
    index_geometry = landmarks_gdf.columns.get_loc("geometry")+1
    index_buildingID = landmarks_gdf.columns.get_loc("buildingID")+1
    
    # The queue stores priority, node, cost to reach, and parent.
    # Uses Python heapq to keep in priority order.
    # Add a counter to the queue to prevent the underlying heap from
    # attempting to compare the nodes themselves. The hash breaks ties in the
    # priority and is guarenteed unique for all nodes in the graph.
    c = count()
    queue = [(0, next(c), source, 0, None)] # openSet
    
    enqueued = {}
    # Maps explored nodes to parent closest to the source.
    explored = {}

    while queue:
        # Pop the smallest item from queue.
        _, __, current_node, cost_so_far, coming_from = pop(queue)
    
        if current_node == target: #construct path
            path = [current_node]
            node = coming_from
            while node is not None:
                path.append(node)
                node = explored[node]
            path.reverse()
            return path
        

        if current_node in explored: continue
        explored[current_node] = coming_from
        for nodeTarget, edgeAtt in G[current_node].items():
            
            if nodeTarget in explored: continue

            index_landmarks_at_node = nodes_gdf['landmarks'][nodes_gdf['nodeID'] == G.node[nodeTarget]['nodeID']].tolist()[0]
            
            
            if (isinstance(index_landmarks_at_node, list) == False): lScore_total = 1
            
            else:
                landmarks_at_node = landmarks_gdf[landmarks_gdf.buildingID.isin(index_landmarks_at_node)]

                geometry_node = G.node[nodeTarget]['geometry']

                lScore_total = 0

                for row in landmarks_at_node.itertuples():

                    cumulative_AdvanceVis = 0    
                    distance_travelled = 0
                    nodeTo =  nodeTarget
                    nodeFrom = current_node

                    while ((nodeFrom is not None) & (distance_travelled <= 200)):

                            nodeColumn = str(nodeFrom)
                            visible = visibility_matrix[visibility_matrix['buildingID']==row[index_buildingID]][nodeColumn].tolist()[0]
                            
                            if visible == True: cumulative_AdvanceVis = cumulative_AdvanceVis + G[nodeFrom][nodeTo]['length'] 
                            distance_travelled = distance_travelled + G[nodeFrom][nodeTo]['length']
                            nodeTo = nodeFrom
                            nodeFrom = explored[nodeFrom]

                    aV = cumulative_AdvanceVis/distance_travelled
                    tmp = row[index_lScore] * aV

                    if tmp > lScore_total: lScore_total = tmp
            
            node_DD_score = G.node[nodeTarget]['DD_sc']
            tentative = cost_so_far + edgeAtt.get(weight, 1) + (1-lScore_total) + node_DD_score
            
            if nodeTarget in enqueued:
                qcost, heuristic = enqueued[nodeTarget]
                if qcost <= cost_so_far: continue

            else: heuristic = 0
            
            enqueued[nodeTarget] = cost_so_far, heuristic
            push(queue, (cost_so_far + heuristic, next(c), nodeTarget, cost_so_far, current_node))

    raise nx.NetworkXNoPath("Node %s not reachable from %s" % (source, target))

In [ ]:
def ec(G, normalized = False, weight = None, seed=None):

    Eb = dict.fromkeys(G.edges(), 0.0)
    nodesT = list(dict.fromkeys(G.nodes()))
    processed = []
    
    for s in     nodesT:

        for t in (t for t in nodesT if s != t):

            if (((s,t) in processed) & ((t,s) in processed)): continue
            sp = astar_path(G, s, t, nodes, landmarks, weight=weight)
            
            for i, n in enumerate(sp):
                if i == 0: continue
                else: path = (sp[i], sp[i-1])

                if path in Eb: path = path
                else: path = (sp[i-1], sp[i])

                Eb[path] += 1
                
    return Eb

In [ ]:
%%time
t = ec(G, normalized = False, weight = 'lenght_sc')

In [ ]:
def _single_source_dijkstra_path_basic(G, s, weight):
    # modified from Eppstein
    S = []
    P = {}
    for v in G:
        P[v] = []
    sigma = dict.fromkeys(G, 0.0)    # sigma[v]=0 for v in G
    D = {}
    sigma[s] = 1.0
    push = heappush
    pop = heappop
    seen = {s: 0}
    c = count()
    Q = []   # use Q as heap with (distance,node id) tuples
    push(Q, (0, next(c), s, s))
    while Q:
        (dist, _, pred, v) = pop(Q)
        if v in D:
            continue  # already searched this node.
        sigma[v] += sigma[pred]  # count paths
        S.append(v)
        D[v] = dist
        for w, edgedata in G[v].items():
            vw_dist = dist + edgedata.get(weight, 1)
            if w not in D and (w not in seen or vw_dist < seen[w]):
                seen[w] = vw_dist
                push(Q, (vw_dist, next(c), v, w))
                sigma[w] = 0.0
                P[w] = [v]
            elif vw_dist == seen[w]:  # handle equal paths
                sigma[w] += sigma[v]
                P[w].append(v)
    return S, P, sigma

def et(G, k=None, normalized=True, weight=None):
    betweenness = dict.fromkeys(G, 0.0)  # b[v]=0 for v in G
    # b[e]=0 for e in G.edges()
    betweenness.update(dict.fromkeys(G.edges(), 0.0))
    
    nodes = G

    for s in nodes:
        S, P, sigma = _single_source_dijkstra_path_basic(G, s, weight)
        print(S, '------------------')
        print(P, '------------------')
        print(sigma, '------------------')
        # accumulation
        betweenness = _accumulate_edges(betweenness, S, P, sigma, s)
        break
    # rescaling
    for n in G:  # remove nodes to only return edges
        del betweenness[n]
    
    betweenness = _rescale_e(betweenness, len(G), normalized=normalized,
                             directed=G.is_directed())
    return betweenness

In [ ]:
et(G, normalized = True, weight = 'length')

In [ ]:
Eb = dict.fromkeys(G.edges(), 0.0)

In [ ]:
edges = []
for t in G.edges(): edges.append(t)
dict_edges = {k: 0 for v, k in enumerate(edges)}


In [ ]:
sp = nx.astar_path(G, 1, 2, weight='length')
sp[1]

In [ ]:
Lb = nx.edge_betweenness_centrality(Ng, weight='land_sc', normalized=False) 

In [ ]:
ELb = nx.edge_betweenness_centrality(Ng, weight='dist&land', normalized=False)

In [ ]:
Ab_df = ic.dict_to_df([Lb, ELb, edge_id], ["Lb", "ELb","streetID"])
paths_tmp = pd.merge(paths, edges_df, left_on = 'streetID', right_on = 'streetID', how='left')

In [ ]:
edge_id={}
for i, g in Ng.edges(): edge_id[(i,g)]=Ng[i][g]['streetID']

edges_df = to_df([Lb, ELb, edge_id], ["Lb", "ELb","streetID"])
edges_df.streetID = edges_df.streetID.astype(int)

In [ ]:
ic.plotGDF(paths_tmp,'Eb', 'Euclidean Betweenness')
ic.plotGDF(paths_tmp,'Lb', 'Landmark Betweenness')
ic.plotGDF(paths_tmp,'ELb', 'Landmark and distance Betweenness')

In [ ]:
f, ax = plt.subplots(1, figsize=(10, 10))
nodes.plot(ax=ax, column='dist', cmap='OrRd', scheme='Fisher_Jenks', markersize=5)
plt.axis('equal')
ax.set_axis_off()

## Dual graph

In [ ]:
nodes_dual = gpd.read_file(folder_ouptut+city_name+'_nodesDual.shp', driver='ESRI Shapefile')
edges_dual = gpd.read_file(folder_ouptut+city_name+'_edgesDual.shp', driver='ESRI Shapefile')

In [ ]:
# assigning landmark scores to links between centroids (this is artificial)

edges_dual.u = edges_dual.u.astype(int)
edges_dual.v = edges_dual.v.astype(int)
edges_dual.key = edges_dual.key.astype(int)

for row in edges_dual.itertuples():
    landmarkness_u = paths['land'][paths.streetID==row[1]].loc[row[1]] #row[1]==u
    landmarkness_v = paths['land'][paths.streetID==row[2]].loc[row[2]] #row[2]==v
    landmarkness = landmarkness_u+landmarkness_v
    edges_dual.set_value(row[0], 'land', landmarkness)

In [ ]:
ic.scaling_columnDF(edges_dual, 'rad')
ic.scaling(edges_dual, 'land') 
edges_dual['angle&land'] = edges_dual['rad_sc']+edges_dual['land_sc']

In [ ]:
nodes_dual.gdf_name = 'Dual_list'
Gr = ox.gdfs_to_graph(nodes_dual, edges_dual)
    
n = Gr.nodes()
pos = {}
    
print(len(n))
for l, item in enumerate(n): pos[l] = (n[l]['x'],n[l]['y'],n[l]['streetID'])
        
DG = nx.Graph() #Empty graph
DG = DG.to_undirected()
DG.add_nodes_from(pos.keys()) #Add nodes preserving coordinates
    
for i, item in enumerate(DG.nodes()):
    DG.node[i]['x']=pos[i][0]
    DG.node[i]['y']=pos[i][1]
    DG.node[i]['streetID']=pos[i][2]
        
for i, item in enumerate(Gr.edges()):
    DG.add_edge(item[0], item[1])
    DG[item[0]][item[1]]['angle&land'] = Gr[item[0]][item[1]][0]['angle&land']

In [ ]:
ALb = nx.betweenness_centrality(DG, weight='angle&land', normalized=False)

In [ ]:
def id_dict(ed, graph):
    
    view = ed.items()
    ed_list = list(view)
    ed_dict = {}

    for p in ed_list:
        ed_dict[graph.node[p[0]]['streetID']]=p[1] #streetID and Edge betweenness
        
    return(ed_dict)

In [ ]:
ALb_dict = id_dict(ALb, DG)
ALb_df = to_df([ALb_dict], ["ALb"])

In [ ]:
paths_tmp = pd.merge(paths_tmp, ALb_df, left_on="streetID", right_index=True, how='left')

In [ ]:
plot(paths_tmp,'Ab', 'Angular Betweenness')
plot(paths_tmp,'ALb', 'Angular-Landmark Betweenness')

In [ ]:
paths.to_file(folder_ouptut+city_name+'_experiment_paths.shp', driver='ESRI Shapefile')

In [ ]:
nodes.drop(['visible_landmarks'], axis=1).to_file(folder_ouptut+city_name+'_experiment_nodes.shp', driver='ESRI Shapefile')

In [17]:
pedestrians = pd.read_csv("C:/Users/g_filo01/sciebo/GIS Data/Simulation/London_pedestrian_counts_27-08.csv")

In [18]:
pp = gpd.read_file("C:/Users/g_filo01/sciebo/Scripts/Image of the City/Outputs/London/London_pedestrians_count.shp")

In [19]:
pp2 = pp[['geometry', 'streetID']].copy()

In [20]:
pp2 = pd.merge(pp2, pedestrians, left_on="streetID", right_on="streetID", how='left')

In [15]:
pp2.head()

,geometry,streetID,euclidean,topological,angular,euclideanLand,topologicalLand,angularLand,landmark
0,LINESTRING (528421.0000001118 180948.999999571...,0,2,17,6,2,19,3,23
1,LINESTRING (533740.0500001372 181263.939999453...,1,30,450,118,10,408,93,535
2,LINESTRING (533835.3200001377 180907.189999451...,2,28,214,119,10,196,113,265
3,LINESTRING (528136.0000001106 180780.999999577...,3,79,263,37,17,511,36,475
4,LINESTRING (532764.1500001322 180312.289999477...,4,426,322,603,491,307,627,314


In [21]:
pp2.to_file(folder_ouptut+city_name+'_pedestrians_count.shp', driver='ESRI Shapefile')

In [ ]:
pedestrians = pd.read_csv("C:/Users/g_filo01/sciebo/GIS Data/Simulation/pedestrians_eucl.csv")

In [ ]:
pedestrians.dtypes

In [ ]:
colPed = ["euclidean", "topological", "angular", "landmark", "euclideanLand", "topologicalLand", "angularLand"]

In [ ]:
paths_data = pd.merge(paths_tmp, pedestrians, left_on="streetID", right_on="streetID", how='left')

In [ ]:
paths_data['euclideanDiff']=paths_data['euclidean']-paths_data['euclideanLand']

In [ ]:
len(paths_data)

In [ ]:
plotGDF(paths_data,"euclidean","euclidean")
# plotGDF(paths_data,"topological","topological")
# plotGDF(paths_data,"angular","angular")
plotGDF(paths_data,"euclideanLand","euclideanLand")
plotGDF(paths_data,"euclideanDiff","diff")
# plotGDF(paths_data,"topologicalLand","topologicalLand")
# plotGDF(paths_data,"angularLand","angularLand")
# plotGDF(paths_data,"landmark","landmark")

In [ ]:
for i in colPed:
    print(i)
    plotGDF(paths_data, colPed, colPed)

In [ ]:
plot_grad_GDF(paths, 'length_sc', 'll')

In [ ]:
Point(mapping(landmarks.loc[5].geometry)['coordinates'][0][0]).intersects(landmarks.geometry)

In [ ]:
for neighbor, w in G[2].items():
    print(neighbor, " lll ",w)
    print(G.node[neighbor]['DD_sc'])
    
    list landmarks
    print
    
    print(G.node[neighbor]['DE_sc'])
    

In [ ]:
astar_path(G, 1, 2, landmarks, weight='length')

In [24]:
pp = gpd.read_file("C:/Users/g_filo01/sciebo/GIS Data/Torino/Turin_edges_data.shp")

In [21]:
pp.dtypes

name        object
osmid       object
edgeID       int64
name_2      object
num         object
perc        object
geometry    object
dtype: object

In [27]:
pp.to_file(folder_ouptut+city_name+'Turin_edges_data.shp', driver='ESRI Shapefile')